In [18]:
# Basic import
import os
import sys
import json
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [19]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

def writeLog(row):
    with open('log.txt', 'a') as outfile:
        outfile.write(row + '\n')

def getErrMsg(e):
    error_class = e.__class__.__name__ #取得錯誤類型
    detail = e.args[0] #取得詳細內容
    errMsg = "[{}] {}".format(error_class, detail)
    return errMsg

In [20]:
DIR = './Data/ours/DMF_data/'

# Load data

In [21]:
usr_test_amount = 150
movie_test_amount = 32

In [22]:
test_idx = read_json(DIR + 'test_idx.json')
print('test_idx:', test_idx)

test_t = read_json(DIR + 'test_t.json')
test_f = read_json(DIR + 'test_f.json')
print(len(test_t), len(test_f))

RS = np.load(DIR + 'testRS.npy')[test_idx, :]
print('RS:', RS.shape)

test_idx: [13, 51, 54, 61, 65, 88, 93, 96, 114, 130, 135, 142, 146, 161, 163, 178, 186, 189, 191, 198, 206, 209, 224, 228, 255, 283, 285, 292, 313, 318, 326, 327, 333, 334, 350, 393, 407, 429, 432, 435, 440, 447, 449, 451, 457, 466, 469, 476, 501, 505, 514, 538, 541, 542, 546, 548, 552, 563, 569, 592, 600, 644, 646, 664, 689, 696, 704, 727, 735, 740, 741, 747, 758, 775, 777, 778, 781, 788, 810, 817, 821, 859, 864, 865, 877, 919, 928, 939, 940, 946, 958, 1010, 1022, 1034, 1043, 1083, 1093, 1098, 1103, 1116, 1130, 1133, 1140, 1149, 1161, 1182, 1195, 1197, 1206, 1209, 1220, 1221, 1232, 1236, 1247, 1266, 1285, 1287, 1300, 1301, 1309, 1310, 1316, 1327, 1330, 1342, 1354, 1372, 1385, 1393, 1399, 1402, 1409, 1429, 1436, 1437, 1442, 1466, 1470, 1493, 1494, 1508, 1516, 1518, 1525, 1529, 1547, 1554, 1563, 1573]
150 150
RS: (150, 165)


In [23]:
def F1_score(prec,rec):
    f1 = 2*((prec*rec)/(prec+rec))
    return f1

def topN(RSls, n):
    maxn = np.argsort(RSls)[::-1][:n]
    return maxn

def allSortPrepare(testRS):
    all_sort = []

    for i in range(usr_test_amount):
        all_sort.append(topN(list(testRS[i]),len(testRS[i])))

    all_sort = np.asarray(all_sort)
    print(all_sort.shape)
    return all_sort

def DCG(prec_list): #找出前n名的[1,1,1,0,...]
    dcg = 0
    for i in range(len(prec_list)):
        dcg += (2**prec_list[i]-1)/math.log2(i+2)
    return dcg

def NDCG(target, testRS, num_ndcg, all_sort): #target是真正的喜好
    total_ndcg = 0
    
    for m in range(usr_test_amount): # the number of testing users
        idcg = DCG(target[m][:num_ndcg])
        
        pre_list = []
        for s in all_sort[m][:num_ndcg]:
            #print(m,s,target[m][s])
            pre_list.append(target[m][s]) #把prec_list 的 score加進去
        
        dcg = DCG(pre_list)
        ndcg = dcg/idcg
        total_ndcg += ndcg
        
    avg_ndcg = total_ndcg/usr_test_amount
    return avg_ndcg

from sklearn.metrics import average_precision_score

def MAP(target,testRS):
    total_prec = 0
    for u in range(usr_test_amount):
        y_true = target[u]
        y_scores = testRS[u]
        total_prec += average_precision_score(y_true, y_scores)
        
    Map_value = total_prec/usr_test_amount
    
    return Map_value

In [24]:
def metrics(testRS, target, sumtarget, all_sort):
    print('\n==============================\n')
    # Top N
    N = [1, 5]
    correct = 0

    for n in N:
        print('Top', n)
        correct = 0

        for i in range(len(testRS)):
            topn = topN(testRS[i], n)
            sum_target = int(np.sum(target[i]))

            TP = 0
            for i in topn:
                if i < sum_target:
                    TP += 1

            correct += TP

        prec = correct/(len(testRS)*n) #150*n
        recall = correct/sumtarget

        print('prec:', prec)
        print('recall:', recall)
        print('F1_score:', F1_score(prec, recall))
        print('*****')

    print('\n==============================\n')

    # NDCG
    num_ndcgs = [10]
    for num_ndcg in num_ndcgs:
        print('NDCG@', num_ndcg)
        print('NDCG score:', NDCG(target, testRS, num_ndcg, all_sort))
        print('*****')

    print('\n==============================\n')

    # MAP
    print('MAP:', MAP(target,testRS))
    print('\n==============================\n')

In [26]:
#取出test的資料
print(RS.shape)

testRS = np.zeros((usr_test_amount, movie_test_amount)) #shape 150 * 32
target = np.zeros((usr_test_amount, movie_test_amount)) #shape 150 * 32

for z in range(usr_test_amount):
    user_id = test_idx[z]
#     print('ID:', user_id)
    
    # positive target YouTuber list
    youtube_t = test_t[z] 
    # not target YouTuber list
    youtube_f = test_f[z]

#     print(user_id)
#     print(youtube_t)
#     print(youtube_f)

    #前面放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1

    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

#     print(testRS[z])
#     print(target[z])
#     print('==============================')

print(target.shape, testRS.shape)
sumtarget = np.sum(target)
print('num of positive data in testing:', sumtarget) # whole matrix: 4800

# for metrics
metrics(testRS, target, sumtarget, allSortPrepare(testRS))

(150, 165)
(150, 32) (150, 32)
num of positive data in testing: 1078.0
(150, 32)


Top 1
prec: 0.02
recall: 0.0027829313543599257
F1_score: 0.004885993485342019
*****
Top 5
prec: 0.424
recall: 0.2949907235621521
F1_score: 0.34792122538293213
*****


NDCG@ 10
NDCG score: 0.32087573009539794
*****


MAP: 0.19669504338863728


